In [49]:
import logging, sys
import pandas as pd
import sqlite3

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)],
    force=True,   # important dans notebooks (reconfigure)
)

In [50]:
import hiring_compass_au.ingestion.gmail.mail_index as mail_index
import hiring_compass_au.storage.mail_store as mail_store
import hiring_compass_au.workspace as workspace

ws = workspace.get_workspace()
db_path = ws.db_path

In [65]:
with mail_store.get_connection(db_path) as conn:
    emails = pd.read_sql_query(
        "SELECT * FROM emails",
        conn,
    )
    
emails.head()

,message_id,thread_id,from_email,subject,internal_date_ms,received_at,indexed_at,fetched_at,parsed_at,status,error,html_raw,hit_extract_count,persist_count,intended_use,template,parser_name,parser_version,parsed_confidence
0,19c1b5c999b4eb3b,19c1b5c999b4eb3b,jobmail@s.seek.com.au,20 new jobs for machine learning engineer,1769985577000,2026-02-01T22:39:37+00:00,2026-02-02T19:23:48+00:00,2026-02-04T18:51:44+00:00,2026-02-11T19:16:04+00:00,parsed,None,"<!doctype html><html lang=""und"" dir=""auto"" xml...",23,0,prod,None,seek_mail_parser,v1,100
1,19c1b10c30e076a7,19c1b10c30e076a7,jobmail@s.seek.com.au,20 new jobs for data analyst,1769980608000,2026-02-01T21:16:48+00:00,2026-02-02T19:23:48+00:00,2026-02-04T18:51:44+00:00,2026-02-11T19:16:04+00:00,parsed,None,"<!doctype html><html lang=""und"" dir=""auto"" xml...",23,0,prod,None,seek_mail_parser,v1,100
2,19c1b0cf11f33578,19c1b0cf11f33578,jobmail@s.seek.com.au,20 new jobs for data engineer,1769980358000,2026-02-01T21:12:38+00:00,2026-02-02T19:23:48+00:00,2026-02-04T18:51:44+00:00,2026-02-11T19:16:04+00:00,parsed,None,"<!doctype html><html lang=""und"" dir=""auto"" xml...",23,0,prod,None,seek_mail_parser,v1,99
3,19c1afb48779de5c,19c1afb48779de5c,jobmail@s.seek.com.au,20 new jobs for data scientist,1769979200000,2026-02-01T20:53:20+00:00,2026-02-02T19:23:48+00:00,2026-02-04T18:51:44+00:00,2026-02-11T19:16:04+00:00,parsed,None,"<!doctype html><html lang=""und"" dir=""auto"" xml...",23,0,prod,None,seek_mail_parser,v1,100
4,19c1add3cf3e11ec,19c1add3cf3e11ec,jobmail@s.seek.com.au,5 new jobs for ml engineer,1769977231000,2026-02-01T20:20:31+00:00,2026-02-02T19:23:48+00:00,2026-02-04T18:51:44+00:00,2026-02-11T19:16:04+00:00,parsed,None,"<!doctype html><html lang=""und"" dir=""auto"" xml...",8,0,prod,None,seek_mail_parser,v1,100


In [57]:
emails["status"].value_counts()

NameError: name 'emails' is not defined

In [67]:
with mail_store.get_connection(db_path) as conn:
    job_hits = pd.read_sql_query(
        "SELECT * FROM email_job_hits",
        conn,
    )

job_hits

,hit_id,message_id,fingerprint,template,title,company,suburb,city,state,location_raw,...,promote_status,promote_reason,job_id,canonical_url,canonical_status,http_status,attempt_count,next_retry_at,last_attempt_at,canon_error
0,1,19becf1df76aaeee,27bbe6cffab97a73,None,Process Engineer - Laboratory,CSL Limited,None,Melbourne,VIC,Melbourne VIC,...,pending,None,89853693,https://www.seek.com.au/job/89853693,ok,302,1,None,2026-02-11T19:16:05+00:00,None
1,2,19becf1df76aaeee,dbfacfa61d609f5d,None,"Principal AI / Data Engineer | Databricks, Spa...",DyFlex Solutions,None,Sydney,NSW,Sydney NSW,...,pending,None,89859151,https://www.seek.com.au/job/89859151,ok,302,1,None,2026-02-11T19:16:06+00:00,None
2,3,19becf1df76aaeee,3f04fb808a692181,None,Project Engineer - Food Manufacturing Melbourne,Metromotion Pty Ltd,Mount Waverley,Melbourne,VIC,"Mount Waverley, Melbourne VIC",...,pending,None,89899915,https://www.seek.com.au/job/89899915,ok,302,1,None,2026-02-11T19:16:06+00:00,None
3,4,19becf1df76aaeee,a3d47065f806b918,None,Senior Machine Learning Engineer,The Onset,None,Sydney,NSW,Sydney NSW,...,pending,None,89855926,https://www.seek.com.au/job/89855926,ok,302,1,None,2026-02-11T19:16:07+00:00,None
4,5,19becf1df76aaeee,8f2d5b1b7cf638bd,None,Senior AI Engineer - ASX200 Enterprise,Galileo Search,None,Sydney,NSW,Sydney NSW,...,pending,None,89866885,https://www.seek.com.au/job/89866885,ok,302,1,None,2026-02-11T19:16:07+00:00,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2034,2035,19c49b5cf1b08da8,64a5bf97728d6cfb,None,Head of Software Engineering (Core Engineering),BrainEye,None,Sydney,NSW,Sydney NSW,...,pending,None,90205789,https://www.seek.com.au/job/90205789,ok,302,1,None,2026-02-11T19:59:44+00:00,None
2035,2036,19c49b5cf1b08da8,e100213d992130fb,None,Data Science Analyst,Medibank,None,Melbourne,VIC,Melbourne VIC,...,pending,None,90207496,https://www.seek.com.au/job/90207496,ok,302,1,None,2026-02-11T19:59:45+00:00,None
2036,2037,19c49b5cf1b08da8,b0ec623cd87eb558,None,Machine Learning Engineer,The Onset,None,Sydney,NSW,Sydney NSW,...,pending,None,90194950,https://www.seek.com.au/job/90194950,ok,302,1,None,2026-02-11T19:59:45+00:00,None
2037,2038,19c49b5cf1b08da8,ca275477b68a8752,None,Junior Machine Learning & Computer Vision Engi...,GaardTech,Luscombe,Gold Coast,QLD,"Luscombe, Gold Coast QLD",...,pending,None,90048237,https://www.seek.com.au/job/90048237,ok,302,1,None,2026-02-11T19:59:46+00:00,None


In [69]:
job_hits[['title','company','location_raw','salary_min','salary_max']].

,title,company,location_raw,salary_min,salary_max
0,Process Engineer - Laboratory,CSL Limited,Melbourne VIC,NaN,NaN
1,"Principal AI / Data Engineer | Databricks, Spa...",DyFlex Solutions,Sydney NSW,NaN,NaN
2,Project Engineer - Food Manufacturing Melbourne,Metromotion Pty Ltd,"Mount Waverley, Melbourne VIC",NaN,NaN
3,Senior Machine Learning Engineer,The Onset,Sydney NSW,165000.0,165000.0
4,Senior AI Engineer - ASX200 Enterprise,Galileo Search,Sydney NSW,NaN,NaN
...,...,...,...,...,...
2034,Head of Software Engineering (Core Engineering),BrainEye,Sydney NSW,200000.0,220000.0
2035,Data Science Analyst,Medibank,Melbourne VIC,NaN,NaN
2036,Machine Learning Engineer,The Onset,Sydney NSW,150000.0,150000.0
2037,Junior Machine Learning & Computer Vision Engi...,GaardTech,"Luscombe, Gold Coast QLD",40.0,50.0


In [21]:
with mail_store.get_connection(db_path) as conn:
    df = pd.read_sql_query(
        "SELECT hit_id,message_id,title, company, location_raw, salary_text, debug_lines FROM email_job_hits",
        conn,
    )

df


,hit_id,message_id,title,company,location_raw,salary_text,debug_lines
0,1,19becf1df76aaeee,Process Engineer - Laboratory,CSL Limited,Melbourne VIC,None,"[""Process Engineer - Laboratory CSL Limited Me..."
1,2,19becf1df76aaeee,"Principal AI / Data Engineer | Databricks, Spa...",DyFlex Solutions,Sydney NSW,None,"[""Principal AI / Data Engineer | Databricks, S..."
2,3,19becf1df76aaeee,Project Engineer - Food Manufacturing Melbourne,Metromotion Pty Ltd,"Mount Waverley, Melbourne VIC",None,"[""Project Engineer - Food Manufacturing Melbou..."
3,4,19becf1df76aaeee,Senior Machine Learning Engineer,The Onset,Sydney NSW,None,"[""Senior Machine Learning Engineer The Onset S..."
4,5,19becf1df76aaeee,Senior AI Engineer - ASX200 Enterprise,Galileo Search,Sydney NSW,None,"[""Senior AI Engineer - ASX200 Enterprise Galil..."
...,...,...,...,...,...,...,...
1930,1931,19c448f776033772,Data Engineer,Randstad Digital,Canberra ACT,None,"[""Data Engineer Randstad Digital Canberra ACT""..."
1931,1932,19c448f776033772,AI Analyst,Preacta Recruitment,Sydney NSW,None,"[""AI Analyst Preacta Recruitment Sydney NSW"", ..."
1932,1933,19c448f776033772,Machine Learning Engineer,Posted on 1 Feb 2026,Sydney NSW,None,"[""Posted on 1 Feb 2026 Machine Learning Engine..."
1933,1934,19c448f776033772,Junior Machine Learning & Computer Vision Engi...,Posted on 31 Jan 2026,"Luscombe, Gold Coast QLD",None,"[""Posted on 31 Jan 2026"", ""Junior Machine Lear..."


In [44]:
df[df["company"] == "Davidson"]

,hit_id,message_id,title,company,location_raw,salary_text,debug_lines
474,475,19c014d2c2ad91a0,Azure Data Engineer,Davidson,Brisbane QLD,None,"[""Azure Data Engineer Davidson Brisbane QLD"", ..."
583,584,19c067389fa0e04d,Azure Data Engineer,Davidson,Brisbane QLD,None,"[""Azure Data Engineer Davidson Brisbane QLD"", ..."
1070,1071,19c20039aa4b25e1,ML / AI Engineer,Davidson,Brisbane QLD,None,"[""ML / AI Engineer Davidson Brisbane QLD"", ""ML..."
1093,1094,19c2021a687b6776,ML / AI Engineer,Davidson,Brisbane QLD,None,"[""ML / AI Engineer Davidson Brisbane QLD"", ""ML..."
1115,1116,19c20334f47fef64,ML / AI Engineer,Davidson,Brisbane QLD,None,"[""ML / AI Engineer Davidson Brisbane QLD"", ""ML..."
1152,1153,19c2082f36ea410b,ML / AI Engineer,Davidson,Brisbane QLD,None,"[""ML / AI Engineer Davidson Brisbane QLD"", ""ML..."
1174,1175,19c2529f59a904c6,ML / AI Engineer,Davidson,Brisbane QLD,None,"[""ML / AI Engineer Davidson Brisbane QLD"", ""ML..."
1267,1268,19c25a94e8538c51,ML / AI Engineer,Davidson,Brisbane QLD,None,"[""ML / AI Engineer Davidson Brisbane QLD"", ""ML..."
1287,1288,19c2a504f5186ef4,ML / AI Engineer,Davidson,Brisbane QLD,None,"[""ML / AI Engineer Davidson Brisbane QLD"", ""ML..."
1382,1383,19c2acfa924a9516,ML / AI Engineer,Davidson,Brisbane QLD,None,"[""ML / AI Engineer Davidson Brisbane QLD"", ""ML..."


In [48]:
df[df["company"] == "NES Fircroft"]

,hit_id,message_id,title,company,location_raw,salary_text,debug_lines
1908,1909,19c4443a42828c4e,Supply Chain Analyst,NES Fircroft,Perth WA,None,"[""Supply Chain Analyst NES Fircroft Perth WA M..."


In [37]:
df.loc[df["hit_id"] == 1935, "debug_lines"].iloc[0]

'["Posted on 2 Feb 2026 Machine Learning Engineer oOh! North Sydney, Sydney NSW", "Posted on 2 Feb 2026 Machine Learning Engineer oOh! North Sydney, Sydney NSW", "Posted on 2 Feb 2026", "Machine Learning Engineer", "Machine Learning Engineer", "oOh!", "North Sydney, Sydney NSW"]'